# Blog Post Generation using increasingly detailed summaries of sections


## Setup

In [ ]:
!pip install openai


import os
import openai
import re
from pprint import pprint
from getpass import getpass

In [ ]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

## Generating an Outline
Here we generate an outline of the blog post, divided into sections.

In [43]:
MODEL = "gpt-3.5-turbo"

tone = "professional"
synopsis = "The three different types of chocolate, milk, dark, and white. The blog post should be informative and educational and feature a comparison of the health benefits of each type of chocolate."
type_of_content = "blog post"
role_template = f"""You are writer who specialises in outlining the structure of {type_of_content}."""
prompt_template = f""" 
Develop a well-structured outline for a {tone} {type_of_content} about {synopsis}. The outline will be divided into sections, you will output the title of each section and give a three bullet points describing what it will talk about. Make sure each section stands on its own and doesn't repeat things that have been said in other sections. Give your output in the following format: "Section Title:<title> Description:<description>\n".
"""
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": role_template},
        {"role": "user", "content": prompt_template},  ],
    temperature=0,
)

In [66]:
# Parse the response
response_content = response.choices[0].message.content
sections = response_content.split("Section Title:")[1:]
short_summaries = []
for section in sections:
    title, description = section.split("Description:")
    title = title.replace("\n","").strip()
    short_summaries.append((title, description))

In [69]:
for title, description in short_summaries:
    print(f"{title}\n{description}")

Introduction
 Introduce the topic of the blog post and provide a brief overview of the three different types of chocolate.


Milk Chocolate
 Discuss the characteristics and composition of milk chocolate, including its higher sugar and milk content compared to other types of chocolate.
- Explain the process of making milk chocolate and its smooth and creamy texture.
- Highlight popular brands and products that use milk chocolate.


Dark Chocolate
 Explore the features and qualities of dark chocolate, known for its rich flavor and higher cocoa content.
- Discuss the health benefits of dark chocolate, such as its antioxidant properties and potential cardiovascular benefits.
- Explain the different percentages of cocoa in dark chocolate and how it affects taste and health benefits.


White Chocolate
 Examine the unique characteristics of white chocolate, which is technically not a true chocolate due to the absence of cocoa solids.
- Discuss the ingredients used in white chocolate, such as 

## Building more detailed summaries
Now, for each scene we will generate a more detailed summary of the events in that scene. This will summary will then be used in the prompt to generate the script for that scene.

In [64]:
# Summary generation
def generate_section_summary(section_title, section_description, previous_section=None, next_section=None):
    context = ""
    context += f" the previous section was called {previous_section}," if previous_section else ""
    context += f" the section that you are writing is called {section_title},"
    context += f" the next section after this is going to be called {next_section}." if next_section else ""
    summary_role = f"You are a bot who expands brief summaries of a section of a {type_of_content} into bullet pointed summaries that a writer can work from. For context,{context} But you will be given an individual section summary to expand."
    summary_prompt = f"""In BULLET POINTS write a detailed summary for the following section summary:{section_title}\n{section_description}. 
    Ensure that the summary is consistent with and fits into the synopsis of the whole {type_of_content}."""
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": summary_role},
            {"role": "user", "content": summary_prompt},  ],
        temperature=0,
    )
    response_content = response.choices[0].message.content
    section_summary = response_content # TODO: extract the summary from the response if necessary
    return section_summary

## Generating written content
Now we can generate the text content for each section using the detailed summary for that section.

In [49]:
# Script generation
def generate_section_text(section_summary):
    if section_summary is None:
        raise ValueError("section_summary cannot be None")
    script_role = f"You are a {tone} {type_of_content} writer. You will be given a detailed summary of a section of a blog post and will output it in well-written, reader-friendly fashion. If you are unsure of something, leave gaps in the output for another writer to fill in. Do not make up any information."
    script_prompt = f"""The section summary is as follows:\n{section_summary}\n\nWrite the section making sure you follow the points described in the summary. The writing should be engaging, entertaining and {tone} in tone, it can include the use of bullet points and gaps for images if appropriate. The section should be between 100 and 200 words long."""
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": script_role},
            {"role": "user", "content": script_prompt},  ],
        temperature=0,
    )
    response_content = response.choices[0].message.content
    # print(response_content)
    
    return response_content

In [70]:
example_title, example_description = short_summaries[0]
example_summary = generate_section_summary(example_title, example_description, None, short_summaries[1][0])
print(example_summary)

- The introduction of the blog post serves to introduce the topic of chocolate and provide a brief overview of the three different types of chocolate.
- It sets the stage for the rest of the blog post by giving readers a general understanding of what will be discussed.
- The introduction may include a brief history of chocolate and its significance in various cultures.
- It may also mention the popularity of chocolate as a sweet treat and its versatility in various culinary creations.
- The three different types of chocolate that will be explored in the blog post are milk chocolate, dark chocolate, and white chocolate.
- The introduction may touch upon the basic characteristics of each type, such as their ingredients and flavor profiles.
- It may also mention the different uses and preferences for each type of chocolate.
- Overall, the introduction aims to capture the readers' attention and provide a foundation for the subsequent sections that delve deeper into each type of chocolate.


In [65]:
blog_post = ""
i = 0
for section_title, section_description in short_summaries:
    prev_title = None
    next_title = None
    if i > 0:
        prev_title, _ = short_summaries[i-1]
    if i < len(short_summaries) - 1:
        next_title, _ = short_summaries[i+1]
    section_summary = generate_section_summary(section_title, section_description, prev_title, next_title)
    section_text = generate_section_text(f"{section_title}\n{section_description}")
    blog_post += f"## {section_title}\n{section_text}\n\n"
    i += 1

print("Final blog post:")
print(blog_post)

Final blog post:
## Introduction
Welcome to our blog post all about chocolate! In this article, we will be exploring the wonderful world of chocolate and delving into the three different types that exist. Whether you're a chocolate connoisseur or simply have a sweet tooth, this post is sure to satisfy your cravings for knowledge.

Now, let's dive right in and take a look at the three main types of chocolate:

1. Milk Chocolate: This is the most popular type of chocolate and is loved by people of all ages. It is made by combining cocoa solids, cocoa butter, sugar, and milk powder. The result is a smooth and creamy texture with a sweet and milky flavor. Milk chocolate is perfect for those who prefer a milder and less intense chocolate experience.

2. Dark Chocolate: If you're a fan of rich and intense flavors, then dark chocolate is for you. Made with a higher percentage of cocoa solids and less sugar, dark chocolate has a bittersweet taste that is often described as complex and sophisti

In [34]:
sections[0]

'1: Introduction\n- Brief overview of the blog post topic\n- Explanation of the three different types of chocolate\n- Mention of the focus on health benefits\n\n'

## Improving continuity
The sections are good in their own right but they often repeat information from previous sections or generally talk in a way that is clearly ignorant of what has been or will be talked about. To improve this we will need to provide more context to the writer about what has been talked about so far.

### Keeping track of key events
In order to keep track of key events we will introduce a new variable `key_events` which will be a list of the key events that have happened so far in the story. We will add this to the prompt for each scene. To do this we will need to rewrite the `generate_section_text` function.

### Further improvements
These results are quite promising, the script is finally getting longer and roughly follows the initial storyline laid out at the beginning, but there are still some issues: 
- The start of one scene does not always make sense in the context of the previous scene. The scenes can sometimes repeat the events of the previous one, or go in a different direction. This can be improved by making model output better key events after it finishes its script, and in particular, outputting where the scene leaves off.

- Sometimes, the script for a scene continues to generate beyond the scene summary, and into the next scene. This is because the model is not aware of the next scene's story and so it doesn't know it should stop. We can try to fix this by: 
    - adding the next scene's summary to the prompt
    - giving explicit instructions on how each scene should end

    
- The events of scenes which happened further back in the story are not remembered and can lead to incongruous events. This could be improved by adding the key events of the whole story to the prompt. However, we need to be careful we do not "dilute" the prompt too much by adding too much information. Another approach could be to track the status of each character throughout the story and add this to the prompt. For example, if a character is dead, we can add this to the prompt so that the model knows not to include them in the scene.

- The scenes being generated are always a direct continuation of the previous scene. In actual movie scripts, scenes are often not directly related to the previous scene and may jump between several sub-plots, which interweave throughout the story. This creates much more engaging movies. We can try to address this by improving the intial story summary to include a more interesting story structure, however we would then need to more carefully manage which key events are passed to each scene.

Note: We could check that the scene's key events line up to the scene summary each time we generate a script and if there is a mismatch, request a new script generation, possibly with a different temperature.
